<a href="https://colab.research.google.com/github/sgomezb/10k_predictor/blob/main/text_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.2 MB/s eta 0:00:00


In [ ]:
#Para manejo de dataframes
import pandas as pd                     
#Expresiones regulares
import re                               
#Para gráficas
import matplotlib.pyplot as plt       
#Para crear el BoW
from gensim.corpora import Dictionary    
#Procesamiento multiple
import multiprocessing as mp            
import time                             
#Para el modelo LDA
from gensim.models.ldamulticore import LdaMulticore       
#Para el modelo LDA
from gensim.models import CoherenceModel             
#Para stemming
from nltk.stem import PorterStemmer         
#Para la creación del BoW
from gensim.test.utils import common_texts
#Para la creación del BoW
from gensim.corpora.dictionary import Dictionary
#Para la creación del modelo LDA
from gensim.models.ldamulticore import LdaModel
from nltk.stem import WordNetLemmatizer
import PyPDF2

In [ ]:
#configuración en google colab de spark y pyspark
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
path = '/content/gdrive/MyDrive/10k_predictor/data/raw/unstructured/001.pdf'

In [ ]:
def format_number(num):
    return "{:03d}".format(num)

In [ ]:
lista = []
for i in range(1,111):
  id = format_number(i)
  path = '/content/gdrive/MyDrive/10k_predictor/data/raw/unstructured/'+id+'.pdf'
  textopdf =''
  # Abre el archivo PDF en modo de lectura en binario
  with open(path, 'rb') as archivo_pdf:
      # Crea un objeto PdfReader
      lector_pdf = PyPDF2.PdfReader(archivo_pdf)

      # Itera sobre cada página del archivo PDF
      for pagina_actual in lector_pdf.pages:
          # Extrae el texto de la página actual
          texto_pagina_actual = pagina_actual.extract_text()

          # Haz algo con el texto de la página actual
          textopdf += texto_pagina_actual
      lista.append(textopdf)

In [ ]:
csvpath = '/content/gdrive/MyDrive/10k_predictor/data/raw/structured/RawDataSet.csv'

In [ ]:
df = pd.read_csv(csvpath)

In [ ]:
df.head()

,id,ticker,name,eva,eva_spd,mkt_cap,texto
0,1,AAPL UW,APPLE INC,80829.151310,48.159316,2.643060e+12,UNITED STATES\nSECURITIES AND EXCHANGE COMMISS...
1,2,MSFT UW,MICROSOFT CORP,36013.162230,13.581724,2.137120e+12,\nUNITED STATES\nSECURITIES AND EXCHANGE COMM...
2,3,GOOGL UW,ALPHABET INC-CL A,26391.742740,9.339334,1.348900e+12,UNITED STATES\nSECURITIES AND EXCHANGE COMMISS...
3,4,AMZN UW,AMAZON.COM INC,-8289.172578,-2.674548,1.063770e+12,Table of Contents\nUNITED STATES\nSECURITIES A...
4,5,BRK/B UN,BERKSHIRE HATHAWAY INC-CL B,-85871.047610,-12.529865,7.104520e+11,\nUNITED STATES\nSECURITIES AND EXCHANGE COM...


In [ ]:
df = df.rename(columns={'Ticker': 'ticker','Name': 'name','Co Des': 'co_des','WACC Economic Value Added': 'eva','WACC EVA Spd':'eva_spd','Mkt Cap': 'mkt_cap','# Num': 'id'})

In [ ]:
df = df[['id','ticker','name','eva','eva_spd','mkt_cap']]

,id,ticker,name,eva,eva_spd,mkt_cap
0,1,AAPL UW,APPLE INC,80829.151310,48.159316,2.643060e+12
1,2,MSFT UW,MICROSOFT CORP,36013.162230,13.581724,2.137120e+12
2,3,GOOGL UW,ALPHABET INC-CL A,26391.742740,9.339334,1.348900e+12
3,4,AMZN UW,AMAZON.COM INC,-8289.172578,-2.674548,1.063770e+12
4,5,BRK/B UN,BERKSHIRE HATHAWAY INC-CL B,-85871.047610,-12.529865,7.104520e+11
